In [1]:
from pathlib import Path
from datasets import load_dataset
import pandas as pd
import re

# Define paths using pathlib for easier directory creation
DATA_PATH = Path("../data")
OMI_PATH_processed = DATA_PATH / "processed" / "omi-health"
OMI_PATH_raw = DATA_PATH / "raw" / "omi-health"



OMI_PATH_processed.mkdir(parents=True, exist_ok=True)

C:\Users\khal6952\.conda\envs\medical-soap-note-generator\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the dataset
ds_omi_health = load_dataset("omi-health/medical-dialogue-to-soap-summary")
ds_adesouza = load_dataset("adesouza1/soap_notes")

# Save the dataset to disk
ds_omi_health.save_to_disk(OMI_PATH_raw)

Using the latest cached version of the dataset since omi-health/medical-dialogue-to-soap-summary couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\khal6952\.cache\huggingface\datasets\omi-health___medical-dialogue-to-soap-summary\default\0.0.0\9d5d62558de21193e9ba4a7500f42edfe94027c3 (last modified on Thu May  8 15:47:51 2025).
Using the latest cached version of the dataset since adesouza1/soap_notes couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\khal6952\.cache\huggingface\datasets\adesouza1___soap_notes\default\0.0.0\11084a66043aca1cf8809a839d5d43881c229afb (last modified on Thu May  8 15:47:56 2025).
Saving the dataset (1/1 shards): 100%|██████████| 250/250 [00:00<?, ? examples/s]


In [3]:
if 'train' in ds_omi_health:
    train_df_omi = ds_omi_health['train'].to_pandas()

if 'validation' in ds_omi_health:
    val_df_omi = ds_omi_health['validation'].to_pandas()

if 'test' in ds_omi_health:
    test_df_omi = ds_omi_health['test'].to_pandas()

# Check the first few rows of the dataset
train_df_omi.head()

,dialogue,soap,prompt,messages,messages_nosystem
0,"Doctor: Hello, how can I help you today?\nPati...",S: The patient's mother reports that her 13-ye...,Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper..."
1,"Doctor: Hello, what brings you in today?\nPati...","S: The patient, a 21-month-old male, presented...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper..."
2,"Doctor: Hello, how can I help you today?\nPati...","S: Patient reports experiencing fatigue, night...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper..."
3,"Doctor: Hello, Patient D. How are you feeling ...","S: Patient D, a 60-year-old African American m...",Create a medical SOAP summary of this dialogue.,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper..."
4,"Doctor: Hello, I see that you have a history o...","S: The patient, a married woman with a 7-year ...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper..."


In [4]:
train_df_omi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9250 entries, 0 to 9249
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   dialogue           9250 non-null   object
 1   soap               9250 non-null   object
 2   prompt             9250 non-null   object
 3   messages           9250 non-null   object
 4   messages_nosystem  9250 non-null   object
dtypes: object(5)
memory usage: 361.5+ KB


In [5]:
# Let's check one example
train_df_omi.iloc[0]['dialogue']

"Doctor: Hello, how can I help you today?\nPatient: My son has been having some issues with speech and development. He's 13 years old now.\nDoctor: I see. Can you tell me more about his symptoms? Does he have any issues with muscle tone or hypotonia?\nPatient: No, he doesn't have hypotonia. But he has mild to moderate speech and developmental delay, and he's been diagnosed with attention deficit disorder.\nDoctor: Thank you for sharing that information. We'll run some tests, including an MRI, to get a better understanding of your son's condition. \n(After the tests)\nDoctor: The MRI results are in, and I'm glad to say that there are no structural brain anomalies. However, I did notice some physical characteristics. Does your son have any facial features like retrognathia, mild hypertelorism, or a slightly elongated philtrum and thin upper lip?\nPatient: Yes, he has all of those features. His hands are also broad and short. And his feet have mild syndactyly of the second and third toe, 

In [6]:
train_df_omi.iloc[0]['soap']

"S: The patient's mother reports that her 13-year-old son has mild to moderate speech and developmental delays and has been diagnosed with attention deficit disorder. She denies any issues with muscle tone or hypotonia. The patient also exhibits certain physical characteristics, including retrognathia, mild hypertelorism, an elongated philtrum, thin upper lip, broad and short hands, mild syndactyly of the second and third toes, and a sandal gap in both feet.\nO: An MRI of the brain showed no structural anomalies. Whole Exome Sequencing (WES) revealed a de novo frameshift variant Chr1(GRCh37):g.244217335del, NM_205768.2(ZBTB18):c.259del(p.(Leu87Cysfs*21)), indicating a premature termination codon located more than 400 codons upstream of the canonical termination codon.\nA: The primary diagnosis is a genetic disorder associated with the identified frameshift mutation, which likely contributes to the patient's speech and developmental delays and attention deficit disorder. The physical ch

In [7]:
# Event tags extraction
def extract_dialogue_tags(dialogue):
    if pd.isna(dialogue):
        return []  # Return an empty list for missing dialogues

    # Regex to find text enclosed in ( ) or [ ] that appears on its own line.
    # \n : Matches a newline character.
    # \s* : Matches any whitespace character (space, tab, newline, etc.) zero or more times.
    # ( : Start of a capturing group (this is what re.findall will return).
    #   \(.*?\) : Matches anything inside literal parentheses (non-greedy).
    #   | : OR operator.
    #   \[.*?\] : Matches anything inside literal square brackets (non-greedy).
    # ) : End of the capturing group.
    # \s* : Matches any whitespace character after the tag on the same line.
    # (?:\n|$) : Non-capturing group that matches either a newline character or the end of the string.
    # This ensures the tag is essentially on a line by itself.
    pattern = r"\n\s*(\(.*?\)|\[.*?\])\s*(?:\n|$)"
    tags = re.findall(pattern, str(dialogue))
    # convert list to a comma cooperated text
    text = str(tags)
    return text

In [8]:
for index, row in train_df_omi.head().iterrows():
    tags_on_new_lines = extract_dialogue_tags(row['dialogue'])
    print(f"\nTags found on new lines: {tags_on_new_lines}")



Tags found on new lines: ['(After the tests)']

Tags found on new lines: ['[After the tests]', '[After 3 weeks of therapy]']

Tags found on new lines: []

Tags found on new lines: []

Tags found on new lines: []


In [9]:
tags_df = train_df_omi['dialogue'].apply(extract_dialogue_tags).apply(pd.Series)

train_df_omi = pd.concat([train_df_omi, tags_df], axis=1)

train_df_omi = train_df_omi.rename(columns={
    0: 'event_tags'
})

In [10]:
train_df_omi

,dialogue,soap,prompt,messages,messages_nosystem,event_tags
0,"Doctor: Hello, how can I help you today?\nPati...",S: The patient's mother reports that her 13-ye...,Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",['(After the tests)']
1,"Doctor: Hello, what brings you in today?\nPati...","S: The patient, a 21-month-old male, presented...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...","['[After the tests]', '[After 3 weeks of thera..."
2,"Doctor: Hello, how can I help you today?\nPati...","S: Patient reports experiencing fatigue, night...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[]
3,"Doctor: Hello, Patient D. How are you feeling ...","S: Patient D, a 60-year-old African American m...",Create a medical SOAP summary of this dialogue.,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[]
4,"Doctor: Hello, I see that you have a history o...","S: The patient, a married woman with a 7-year ...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[]
...,...,...,...,...,...,...
9245,"Doctor: Hello, I see you're here for a problem...",S: The patient reports difficulty seeing in th...,Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[]
9246,Doctor: Hi there! I see you have brought your ...,"S: The patient, a 3-year-old neutered male Box...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[]
9247,"Doctor: Hello there, how can I help you today?...",S: The patient is a 29-year-old obese male wit...,Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[]
9248,"Doctor: Hello, I understand that you've been t...",S: The patient reports feeling weak but managi...,Create a medical SOAP summary of this dialogue.,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[]


In [11]:
train_df_omi.to_csv(OMI_PATH_processed / 'train_v1.csv', index=False)

In [12]:
# Separate S,O, A, P from soap into different columns
def split_soap(soap):
    # Split the SOAP note into its components
    components = soap.split('\n')
    soap_dict = {}
    for component in components:
        if ':' in component:
            key, value = component.split(':', 1)
            soap_dict[key.strip()] = value.strip()
    return soap_dict

# Apply the function to the dataframe
soap_df = train_df_omi['soap'].apply(split_soap).apply(pd.Series)

# Concatenate the original dataframe with the new soap_df
train_df_omi = pd.concat([train_df_omi, soap_df], axis=1)

# Drop the original soap column
#train_df_omi = train_df_omi.drop(columns=['soap'])

# Rename the columns
train_df_omi = train_df_omi.rename(columns={
    'S': 'subjective',
    'O': 'objective',
    'A': 'assessment',
    'P': 'plan'
})

# Check the first few rows of the updated dataframe
train_df_omi.head()

,dialogue,soap,prompt,messages,messages_nosystem,event_tags,subjective,objective,assessment,plan,...,- Slit lamp examination of the right eye,- Scheimpflug densitometry,- Ears,- Nose,- Oral,- Neck,- Additional findings on the following day,- Laboratory results,- Transvaginal ultrasound showing diffuse fibromatosis with two uterine masses,- Abdominal CT indicated increased uterine volume with two masses
0,"Doctor: Hello, how can I help you today?\nPati...",S: The patient's mother reports that her 13-ye...,Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",['(After the tests)'],The patient's mother reports that her 13-year-...,An MRI of the brain showed no structural anoma...,The primary diagnosis is a genetic disorder as...,The management plan includes regular follow-up...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Doctor: Hello, what brings you in today?\nPati...","S: The patient, a 21-month-old male, presented...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...","['[After the tests]', '[After 3 weeks of thera...","The patient, a 21-month-old male, presented wi...",Hip ultrasound showed no joint effusion. Spine...,Primary diagnosis is Spondylodiscitis with ass...,Initiated broad-spectrum intravenous therapy w...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Doctor: Hello, how can I help you today?\nPati...","S: Patient reports experiencing fatigue, night...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[],"Patient reports experiencing fatigue, night sw...","Vital signs normal. BMI 37.2 kg/m2, weight 263...",The patient presents with symptoms suggestive ...,Continue current medications. Schedule follow-...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Doctor: Hello, Patient D. How are you feeling ...","S: Patient D, a 60-year-old African American m...",Create a medical SOAP summary of this dialogue.,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[],"Patient D, a 60-year-old African American male...",Patient is currently asymptomatic. No physical...,Patient D is at an increased risk for prostate...,Plan to have a detailed conversation about PSA...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Doctor: Hello, I see that you have a history o...","S: The patient, a married woman with a 7-year ...",Create a Medical SOAP note summary from the di...,"[{'role': 'system', 'content': 'You are an exp...","[{'role': 'user', 'content': 'You are an exper...",[],"The patient, a married woman with a 7-year his...",Physical examination confirmed hirsutism and m...,The primary diagnosis is Polycystic Ovarian Sy...,The management plan includes proceeding with i...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Only keep the columns we need
train_df_omi = train_df_omi[['dialogue', 'subjective', 'objective', 'assessment', 'plan']]

# Save the updated dataframe to disk
train_df_omi.to_csv(OMI_PATH_processed / 'train_v2.csv', index=False)

In [17]:
# find the longest dialog in a dataset
def find_longest_dialogue(df, dialogue_column='dialogue'):
    """
    Finds the longest dialogue in a DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame containing the dialogue data.
        dialogue_column (str, optional): The name of the column containing the dialogues.
                                         Defaults to 'dialogue'.

    Returns:
        tuple: A tuple containing:
               - int: The index of the longest dialogue.
               - str: The longest dialogue itself.
    """

    #  Calculate the length of each dialogue
    dialogue_lengths = df[dialogue_column].apply(len)

    # Find the index of the longest dialogue
    longest_dialogue_index = dialogue_lengths.idxmax()

    # Get the longest dialogue
    longest_dialogue = df.loc[longest_dialogue_index, dialogue_column]

    return longest_dialogue_index, len(longest_dialogue)

In [21]:
# Apply the function to find the longest dialogue
longest_index, longest_dialogue_text = find_longest_dialogue(train_df_omi) #   train_df_omi val_df_omi test_df_omi

# Print the results
print(f"The longest dialogue is at index: {longest_index}\n")
print("Longest dialogue text:\n", longest_dialogue_text)

The longest dialogue is at index: 1823

Longest dialogue text:
 3730
